### DecisionTree Best Model  
- RandomizedSearchCV : parmas:{'max_leaf_nodes': 24, 'max_depth': 5} / roc_auc_score_val :  0.826611565425314

- GridSearchCV: params {'max_depth': 6, 'max_leaf_nodes': 29}/ roc_auc_score_val : 0.8418440863076111 


   =>RandomizedSearchCV 에서 찾은 'max_leaf_nodes': 24, 'max_depth': 5 이 들어 갈 수있는 범위를 만들어 준다 


In [1]:
import pandas as pd 
import numpy as np
data =pd.read_csv("preprocessing\cs_data.csv",
                 header=0,
                 index_col=0)

In [2]:
# NA를 전부 없앤 값 = datap
data.dropna(inplace=True)
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
data.shape ,# drop na
data.isnull().sum()


SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
NumberOfDependents                      0
dtype: int64

In [5]:
######################### data 로 dataset 구분 
X= data.drop(columns='SeriousDlqin2yrs')
y= data['SeriousDlqin2yrs']
X.shape , y.shape

((120269, 10), (120269,))

In [6]:
# 1.dataset 구분
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size =0.2, stratify=y ,random_state=0) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=0)

In [9]:
# 2. scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
sc= StandardScaler()
X_train_scaled1=sc.fit_transform(X_train)
X_val_scaled1=sc.transform(X_val)
X_test_scaled1=sc.transform(X_test)


mn = MinMaxScaler()
X_train_scaled2 =mn.fit_transform(X_train)
X_val_scaled2 =mn.transform(X_val)
X_test_scaled2=mn.transform(X_test)

In [ ]:
######## Decision Tree best model 찾기 ############ 

In [13]:
tree.get_params().keys()

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])

In [46]:
#Best DecisionTree param을 RandomizedSearch CV로 찾기
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 

tree=DecisionTreeClassifier(random_state=0)

param={'max_depth': range(5,11),
       'max_leaf_nodes':range(15,30,3)
    
}

rs=RandomizedSearchCV(tree,param, scoring='roc_auc', n_iter=10, cv=5,n_jobs=-1)


In [47]:
rs.fit(X_train_scaled1, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=0),
                   n_jobs=-1,
                   param_distributions={'max_depth': range(5, 11),
                                        'max_leaf_nodes': range(15, 30, 3)},
                   scoring='roc_auc')

In [48]:
rs.best_score_

0.826611565425314

In [49]:
rs.best_params_

{'max_leaf_nodes': 27, 'max_depth': 5}

In [ ]:
#best_score : 0.8267171296392574, best_params :{'max_leaf_nodes': 24, 'max_depth': 5}, best_roc_auc_score : 0.826611565425314

In [50]:
rs.best_score_ ,rs.best_params_,rs.best_estimator_

(0.826611565425314,
 {'max_leaf_nodes': 27, 'max_depth': 5},
 DecisionTreeClassifier(max_depth=5, max_leaf_nodes=27, random_state=0))

In [51]:
best_model=rs.best_estimator_

In [52]:
from sklearn.metrics import roc_curve,plot_roc_curve,roc_auc_score
roc_auc_score(y_val,best_model.predict_proba(X_val_scaled1)[:,1])

0.8406854761602621

In [29]:
#Best DecisionTree param을 GridSearch CV로 찾기
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier 

tree=DecisionTreeClassifier(random_state=0)

param={'max_depth': range(5,11),
       'max_leaf_nodes':range(20,30,3)
    
}

gs=GridSearchCV(tree,param, scoring='roc_auc', cv=5,n_jobs=-1)


In [31]:
gs.fit(X_train_scaled1,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': range(5, 11),
                         'max_leaf_nodes': range(20, 30, 3)},
             scoring='roc_auc')

In [32]:
gs.best_score_

0.8279987279260327

In [36]:
gs.best_params_

{'max_depth': 6, 'max_leaf_nodes': 29}

In [56]:
best_model1=gs.best_estimator_

In [57]:
from sklearn.metrics import roc_curve,plot_roc_curve,roc_auc_score
roc_auc_score(y_val,best_model1.predict_proba(X_val_scaled1)[:,1])

0.8418440863076111

In [ ]:
# score 0.8279987279260327 / params {'max_depth': 6, 'max_leaf_nodes': 29}/ 0.8418440863076111